In [1]:
import pyspark

# create spark session and read in the data
spark = pyspark.sql.SparkSession.builder.appName('demo').getOrCreate()
df = spark.read.option("header","true").csv('data/walmart_stock.csv')

21/11/30 19:36:41 WARN Utils: Your hostname, MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.235 instead (on interface en0)
21/11/30 19:36:41 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/11/30 19:36:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
21/11/30 19:36:44 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
21/11/30 19:36:44 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [2]:
# select useful columns - Open, High, Low, Close
df = df.select(df['Open'].cast('float'),df['High'].cast('float'),df['Low'].cast('float'),df['Close'].cast('float'))

# count number of data entries in the file
entryCount = df.count()
print("total number of entries: ",entryCount)

# find the max stock price
HighPriceDf = df.select(df['High'])
maxPriceDf = HighPriceDf.groupBy()
maxPriceDf = maxPriceDf.max()
maxPriceDf.show()

# find max price fluctuation and final price change in a day
df = df.rdd.map(lambda x : (x[1] - x[2],x[3] - x[0])).toDF(["max_price_fluctuation","final_price_change"])
df.show()

# find average price fluctuation
fluctuationDf = df.select(df["max_price_fluctuation"])
fluctuationDf = fluctuationDf.groupBy()
fluctuationDf = fluctuationDf.avg()
fluctuationDf.show()


total number of entries:  1258
+---------+
|max(High)|
+---------+
|    90.97|
+---------+



+---------------------+--------------------+
|max_price_fluctuation|  final_price_change|
+---------------------+--------------------+
|     1.19000244140625|  0.3600006103515625|
|   0.8799972534179688|                -0.5|
|                 1.25| 0.06999969482421875|
|   0.5800018310546875| -0.4199981689453125|
|   0.6300010681152344| 0.15000152587890625|
|   0.7299995422363281| -0.3899993896484375|
|  0.48999786376953125|  0.3400001525878906|
|   0.5999984741210938|-0.29000091552734375|
|   0.6000022888183594|  0.3600006103515625|
|   0.5900001525878906|-0.02000045776367...|
|  0.37999725341796875| 0.21999740600585938|
|   0.9799995422363281|  0.6800003051757812|
|   0.5800018310546875|  0.2599983215332031|
|    0.470001220703125| 0.09999847412109375|
|                 1.25|  0.6399993896484375|
|   0.5699996948242188| 0.29000091552734375|
|   1.0699996948242188| -0.8299980163574219|
|   0.5799980163574219|-0.15000152587890625|
|    0.970001220703125|  0.8299980163574219|
|   0.9899